In [ ]:
# conversation buffer memory - 모든 대화를 기억
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({"input": "hi!"}, {f"output": "How are you?"})
memory.load_memory_variables({})['history']

'Human: hi!\nAI: How are you?'

In [8]:
# conversation buffer window memory - 최근 대화에 집중.
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
	return_messages=True,
	k=2
)

memory.save_context({"input": "1"}, {"output": "1"})
memory.save_context({"input": "2"}, {"output": "2"})
memory.save_context({"input": "3"}, {"output": "3"})
memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3')]}

In [3]:
# conversation suammry memory - 메모리 활용을 위해 llm 을 사용하는 메모리.
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryMemory(llm=llm)

memory.save_context({"input": "모든 응답은 한국어로 작성해줘."}, {"output": "네, 알겠습니다."})
memory.save_context({"input": "반가워. 나는 대한민국 성남시에서 작은 회사를 운영하는 이 대표라고 해."}, {"output": "반갑습니다. 이 대표님"})
memory.save_context({"input": "AI 를 활용한 소프트웨어 개발에 대해 좋은 아이디어가 없을까?"}, {"output": "더 많은 아이디어를 찾아보겠습니다."})
memory.load_memory_variables({})

{'history': 'The human asks the AI to respond in Korean. The AI acknowledges and agrees to respond in Korean, greeting the human as the CEO of a small company in Seongnam, South Korea. The AI responds politely in Korean, addressing the human as the CEO and agrees to look for more ideas for software development using AI.'}

In [4]:
# conversation summary buffer memory - 최근 대화에 집중하면서도, 이전 대화를 잊어버리는 것이 아니라 요약해서 기억함. (buffer memory + summary memory)
from langchain.memory import ConversationSummaryBufferMemory

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=50, return_messages=True)
memory.save_context({"input": "반가워. 나는 대한민국 성남시에서 작은 회사를 운영하는 이 대표라고 해."}, {"output": "반갑습니다. 이 대표님"})
memory.save_context({"input": "AI 를 활용한 소프트웨어 개발에 대해 좋은 아이디어가 없을까?"}, {"output": "더 많은 아이디어를 찾아보겠습니다."})
memory.load_memory_variables({})

{'history': [SystemMessage(content='The human greets the AI in Korean and introduces themselves as the CEO of a small company in Seongnam, South Korea. The AI responds in Korean and the human asks if there are any good ideas for software development using AI.'),
  AIMessage(content='더 많은 아이디어를 찾아보겠습니다.')]}

In [9]:
# conversation knowledge graph memory - 요약해서 엔티티를 추출하는 메모리
from langchain.memory import ConversationKGMemory

memroy = ConversationKGMemory(llm=llm, return_messages=True)
memory.save_context({"input": "반가워. 나는 대한민국 성남시에서 작은 회사를 운영하는 Mr.Lee라고 해."}, {"output": "반갑습니다. Mr.Lee"})
memory.save_context({"input": "나는 소프트웨어 개발 및 공급을 하고 있어. 최근에는 클라이언트의 홈페이지를 제작했어."}, {"output": "멋진 일을 하고 계시네요."})
memory.save_context({"input": "AI 를 활용한 소프트웨어 개발에 대해 좋은 아이디어가 없을까?"}, {"output": "더 많은 아이디어를 찾아보겠습니다."})
memory.load_memory_variables({"inputs": "Mr.Lee의 최근 작업에 대해서 알려줘."})

{'history': [SystemMessage(content="The human greets the AI in Korean and introduces themselves as the CEO of a small company in Seongnam, South Korea. The AI responds in Korean and the human asks if there are any good ideas for software development using AI. The AI responds that it will look for more ideas to provide and compliments the human's work in website development for clients. Mr. Lee from Seongnam, South Korea, shares that he is involved in software development and recently worked on creating websites for clients. The AI acknowledges Mr. Lee's work and expresses interest in collaborating on AI software development ideas. Mr. Lee expresses his pleasure in meeting the AI and introduces himself as the CEO of a small company in Seongnam, South Korea, mentioning his recent work in website development for clients. The AI compliments Mr. Lee's work and expresses interest in collaborating on AI software development ideas."),
  AIMessage(content='더 많은 아이디어를 찾아보겠습니다.')]}

In [ ]:
# off the shelf chain - 즉시 꺼내쓸 수 있도록 미리 만들어 놓은 체인(?)
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

memory = ConversationSummaryBufferMemory(
	llm=llm,
	max_token_limit=50,
	memory_key="chat_history",
)

# 그냥 질문하면 기억을 못하니, 메모리도 함께 템플릿으로 던진다. memory_key 는 맞춰야 한다.
template = """
	당신은 대화를 통해 사람에게 도움을 주는 AI 입니다.

	{chat_history}
	Human: {question}
	You: 
"""

chain = LLMChain(
	llm=llm,
	memory=memory,
	# prompt=PromptTemplate.from_template("{question}"),
	prompt=PromptTemplate.from_template(template),
	verbose=True
)

In [18]:
# off the shelf chain - 즉시 꺼내쓸 수 있도록 미리 만들어 놓은 체인 (계속)
# chain.predict(question="Hello, my name is Mr.Lee")
chain.predict(question="what is my name?")
# memory.load_memory_variables({})



> Entering new LLMChain chain...
Prompt after formatting:

	당신은 대화를 통해 사람에게 도움을 주는 AI 입니다.

	Human: Hello, my name is Mr.Lee
AI: Hello Mr. Lee! How can I assist you today?
	Human: what is my name?
	You: 


> Finished chain.


'Your name is Mr. Lee.'

In [19]:
# chat based memory 
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

memory = ConversationSummaryBufferMemory(
	llm=llm,
	max_token_limit=50,
	memory_key="chat_history",
	return_messages=True,
)

prompt = ChatPromptTemplate.from_messages([
	("system", "당신은 대화를 통해 사람에게 도움을 주는 AI 입니다."),
	MessagesPlaceholder(variable_name="chat_history"),
	("human", "{question}"),
])

chain = LLMChain(
	llm=llm,
	memory=memory,
	prompt=prompt,
	verbose=True
)

In [21]:
# chat based memory (계속)
# chain.predict(question="Hello, my name is Mr.Lee")
chain.predict(question="what is my name?")
# memory.load_memory_variables({})



> Entering new LLMChain chain...
Prompt after formatting:
System: 당신은 대화를 통해 사람에게 도움을 주는 AI 입니다.
Human: Hello, my name is Mr.Lee
AI: Hello Mr. Lee! How can I assist you today?
Human: what is my name?

> Finished chain.


'Your name is Mr. Lee. How can I assist you further, Mr. Lee?'

In [30]:
# LCEL based memory
from langchain.schema.runnable import RunnablePassthrough

memory = ConversationSummaryBufferMemory(
	llm=llm,
	max_token_limit=50,
	return_messages=True,
)

prompt = ChatPromptTemplate.from_messages([
	("system", "당신은 대화를 통해 사람에게 도움을 주는 AI 입니다."),
	MessagesPlaceholder(variable_name="history"),
	("human", "{question}"),
])

def load_memory(_):
	print(memory.load_memory_variables({})["history"])
	return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
	result = chain.invoke({"question": question})
	memory.save_context(
		{"input": question},
		{"output": result.content},
	)
	print(result)

In [32]:
# LCEL based memory (계속)

# invoke_chain("my name is Mr. Lee")
# invoke_chain("what is my name?")
# invoke_chain("my favorite food is pizza.")
invoke_chain("what is my favorite food?")

[HumanMessage(content='my favorite food is pizza.'), AIMessage(content='Pizza is a popular choice! What do you like most about pizza? The cheese, the toppings, or the crust?')]
content="Your favorite food is pizza! It's great that you enjoy it so much. Do you have a favorite type of pizza or favorite toppings?"
